In [ ]:
# Instalar a biblioteca do cliente do Google e a biblioteca para autenticação
#!pip install --upgrade google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from google.colab import auth
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Nome do arquivo JSON que você fez upload
# Certifique-se de que o nome do arquivo aqui corresponde ao nome do arquivo que você baixou e fez upload
SERVICE_ACCOUNT_FILE = 'mba-usp-469620-7ff99acd80ca.json' # <-- ATUALIZE AQUI COM O NOME DO SEU ARQUIVO JSON

# Adicione o arquivo JSON à pasta raiz do seu Colab ou atualize o caminho abaixo se o upload foi para outra pasta
# O caminho padrão após o upload é geralmente na raiz do ambiente Colab (/content/)

# Carregar as credenciais da conta de serviço
try:
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/spreadsheets.readonly'])
    print("Credenciais carregadas com sucesso!")
except FileNotFoundError:
    print(f"ERRO: O arquivo de credenciais '{SERVICE_ACCOUNT_FILE}' não foi encontrado.")
    print("Por favor, verifique se você fez upload do arquivo JSON para o Colab")
    print("e se o nome do arquivo na variável SERVICE_ACCOUNT_FILE está correto.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar as credenciais: {e}")

# Constrói o serviço Sheets
if 'credentials' in locals():
    service = build('sheets', 'v4', credentials=credentials)
    print("Serviço Google Sheets construído com sucesso!")
else:
    print("Não foi possível construir o serviço Google Sheets devido a erros de credenciais.")

Credenciais carregadas com sucesso!
Serviço Google Sheets construído com sucesso!


In [ ]:
# Passo 3: Ler dados da planilha usando a API do Google Sheets

# Atualize com o ID da sua planilha e o nome da aba (ou range)
# O SPREADSHEET_ID está na URL da sua planilha, entre /d/ e /edit
SPREADSHEET_ID = '1sTiUrfRTddQLssaYDR38ouGwfmkSUP0rVY4ssO2dc2A' # <-- ATUALIZE AQUI COM O ID DA SUA PLANILHA
RANGE_NAME = 'Respostas ao formulário 1!A1:AD42' # <-- ATUALIZE AQUI COM O NOME DA ABA E O RANGE DOS DADOS (ex: Sheet1!A1:D10),

try:
    # Chamada à API para obter os dados
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('Nenhum dado encontrado no range especificado.')
    else:
        print(f'Dados lidos com sucesso! Número de linhas: {len(values)}')
        # Os dados lidos estão na variável 'values' (lista de listas)

except Exception as e:
    print(f"Ocorreu um erro ao ler os dados da planilha: {e}")

Dados lidos com sucesso! Número de linhas: 42


In [ ]:
# Passo 4: Carregar dados em um DataFrame

import pandas as pd
import numpy as np
# A primeira linha da lista 'values' são os cabeçalhos
headers = values[0]
# As linhas restantes são os dados
data = values[1:]

if 'values' in locals() and values:
    # The first row of 'values' are the headers
    headers = values[0]
    # The remaining rows are the data
    data = values[1:]

    # Criar o DataFrame
    df_raw = pd.DataFrame(data, columns=headers)
    print("✅ DataFrame criado com sucesso!")

    # --- Limpar nomes das colunas em df_raw (remover espaços em branco no início/fim) ---
    # This step is still important to handle potential spaces in other columns not explicitly mapped
    df_raw.columns = df_raw.columns.str.strip()
    print("✅ Espaços removidos dos nomes das colunas em df_raw.")

pd.set_option('display.max_columns', None)

display(df_raw.head())

✅ DataFrame criado com sucesso!
✅ Espaços removidos dos nomes das colunas em df_raw.


,Carimbo de data/hora,"Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]","Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]","Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]",Em qual estado da federação você reside?,Há quanto tempo atua na área de tecnologia?,"Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [SI]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]","Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]","Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)",O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.,Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.,Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.,Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área.,"Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)","Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória."
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,,Sao,7 a 10 anos,,,,,,,,,,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None
1,30/05/2025 

In [ ]:
# Mostrar o número de respostas e de perguntas
print(f"Total de respostas (participantes): {df_raw.shape[0]}")
print(f"Total de perguntas (colunas): {df_raw.shape[1]}")
print("\n")

Total de respostas (participantes): 41
Total de perguntas (colunas): 30




In [ ]:
# Analisar a frequência das respostas para o perfil demográfico e profissional
# Usando os nomes originais das colunas, como estão no seu arquivo bruto.
print("### Análise de Frequência - Perfil da Amostra ###")
print("-" * 40)

### Análise de Frequência - Perfil da Amostra ###
----------------------------------------


In [428]:
import pandas as pd
import numpy as np

# --- Passo 1: Ler os dados brutos da sua planilha do Google Sheets ---
# df_raw já carregado da etapa anterior
# print(df_raw.head())

df_raw.columns = df_raw.columns.str.strip()

# --- Mapeamento completo das colunas corrigido ---
column_mapping = {
    'Carimbo de data/hora': 'timestamp',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [até 25 anos]': 'idade_ate_25',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [26 a 35]': 'idade_26_35',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [36 a 45]': 'idade_36_45',
    'Qual a sua idade e nível de escolaridade? \n(Por favor, marque a opção que melhor te representa.) [46 ou mais]': 'idade_46_ou_mais',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Preta]': 'preta',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Parda]': 'parda',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Indígena]': 'indigena',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Amarela]': 'amarela',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Branca]': 'branca',
    'Como você se autodeclara em relação à sua raça/cor e com qual gênero você se identifica?\n(Por favor, marque a opção que melhor te representa.) [Não responder]': 'nao_respondeu_raca',
    'Em qual estado da federação você reside?': 'estado',
    'Há quanto tempo atua na área de tecnologia?': 'tempo_experiencia',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dev / Engenheira de Software]': 'dev_eng_software',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [DevOps]': 'devops',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Dados]': 'dados',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Suporte]': 'suporte',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Infraestrutura]': 'infraestrutura',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [QA]': 'qa',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [SI]': 'si',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [UX/UI]': 'ux_ui',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [PO]': 'po',
    'Qual sua especialidade e senioridade atual?\n(Por favor, para sua principal especialidade atual, marque sua senioridade ) [Negócios]': 'negocios',
    'Se você ingressou na área de tecnologia por vaga afirmativa, qual(is) era(m) o(s) critério(s) de elegibilidade que mais se destacavam? (Marque todas as opções que se aplicam)': 'criterios_vaga_afirmativa',
    'O ingresso por vaga afirmativa teve um impacto positivo na minha trajetória profissional na tecnologia.': 'impacto_vaga_afirmativa',
    'Sinto que minha presença e contribuições são valorizadas no meu ambiente de trabalho.': 'sentimento_valorizacao',
    'Minha empresa (atual ou mais recente que atuava na área de tecnologia) oferece oportunidades de desenvolvimento e crescimento profissional satisfatórias para mulheres negras.': 'oportunidades_desenvolvimento',
    'Já enfrentei barreiras ou preconceito (racial e/ou de gênero) que dificultaram minha trajetória na área. ': 'enfrentou_barreiras',
    'Quais fatores mais te motivam (ou motivariam) a permanecer e progredir em sua carreira na área de tecnologia? (Por favor, selecione até 3 fatores principais)': 'fatores_permanencia',
    'Qual a sua opinião em relação as vagas afirmativas? Se tiver e puder, deixe o relato de sua experiência e o impacto em sua trajetória.': 'opiniao_vagas_afirmativas'
}

# Remover espaços em branco no início/fim das chaves do dicionário column_mapping
column_mapping = {key.strip(): value for key, value in column_mapping.items()}

df = df_raw.rename(columns=column_mapping, errors='ignore')
print("✅ Colunas do DataFrame 'df' renomeadas.")

df.head(0)

✅ Colunas do DataFrame 'df' renomeadas.


,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,estado,tempo_experiencia,dev_eng_software,devops,dados,suporte,infraestrutura,qa,si,ux_ui,po,negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_permanencia,opiniao_vagas_afirmativas,genero,raca


In [429]:
df.head()

,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,estado,tempo_experiencia,dev_eng_software,devops,dados,suporte,infraestrutura,qa,si,ux_ui,po,negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_permanencia,opiniao_vagas_afirmativas,genero,raca
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,,Sao,7 a 10 anos,,,,,,,,,,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None,Mulher(cis),Parda
1,30/05/2025 21:43:56,,Pós-graduação,,,Mulher(cis),,,,,,Pernambuco,1 a 3 anos,Júnior,,,,,,,,,,Não ingressei por vaga afirmativa,Não se aplica (não ingressei por vaga afirmati...,Concordo Totalmente,Concordo Totalmente,Neutro,"Reconhecimento profissional, Remuneração e pac...",Essenciais para melhorar a diversidade no trab...,Mulher(cis),Preta
2,31/05/2025 12:03:02,,Ensino Superior,,,Mulher(cis),,,,,,São Paulo,Menos de 1 ano,Júnior,,,,,,,,,,Interseccional (direcionada a mulheres negras),Concordo,Concordo,Concordo,Discordo,"Desafios técnicos e projetos estimulantes, Rem...",None,Mulher(cis),Preta
3,31/05/2025 17:40:18,,Ensino Superior,,,,Mulher(cis),,,,,São Paulo,Menos de 1 ano,Júnior,,,,,,,,,,"Raça (direcionada a pessoas negras), Gênero (d...",Concordo Totalmente,Concordo,Concordo Totalmente,Discordo,"Desafios técnicos e projetos estimulantes, Fle...",None,Mulher(cis),Parda
4,02/06/2025 10:09:19,Ensino Médio,,,,Mulher(cis),,,,,,São Paulo,1 a 3 anos,,,,,,Estag/Trainee,,,,,Raça (direcionada a pessoas negras),Concordo,Concordo,Concordo Totalmente,Concordo,"Salário competitivo, Desafios técnicos e proje...",As vagas afirmativas são de extrema importânci...,Mulher(cis),Preta


In [430]:
# 2.1 - Funções para consolidação das colunas de matriz
def consolidate_idade_escolaridade(row):
    for col in [c for c in row.index if c.startswith('idade_ate_25') or c.startswith('idade_26_35') or c.startswith('idade_36_45') or c.startswith('idade_46_ou_mais')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            idade = col
            escolaridade = value
            return pd.Series([idade, escolaridade])
            return value
    return np.nan

In [431]:
# 2.1 - Funções para consolidação das colunas de matriz
def consolidate_genero_raca_pair(row):
    for col in [c for c in row.index if c.startswith('preta') or c.startswith('parda') or c.startswith('indigena') or c.startswith('amarela') or c.startswith('branca') or c.startswith('não responder')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            raca = col
            genero = value
            return pd.Series([genero, raca])
    return pd.Series([np.nan, np.nan])

In [432]:
# 2.1 - Funções para consolidação das colunas de matriz
def consolidate_especialidade_senioridade(row):
    for col in [c for c in row.index if c.startswith('dev_eng_software') or c.startswith('devops') or c.startswith('dados') or c.startswith('suporte')
    or c.startswith('infraestrutura') or c.startswith('qa') or c.startswith('si') or c.startswith('ux_ui') or c.startswith('po') or c.startswith('negocios')]:
        value = row[col]
        if pd.notna(value) and str(value).strip() != '':
            especialidade = col
            senioridade = value
            return pd.Series([especialidade, senioridade])
            return value
    return np.nan

**Agrupamento Racial: Criando a Categoria "Mulheres Negras"**
Seu tema central são as "mulheres negras". Na pesquisa em ciências sociais no Brasil, é comum (e metodologicamente embasado pelo IBGE) agrupar as autodeclarações de "Preta" e "Parda" na categoria "Negra". Isso pode fortalecer suas análises, aumentando a representatividade do grupo focal.

In [433]:
def agrupar_raca(raca):
    if raca in ['preta', 'parda']:
        return 'Negra'
    else:
        return raca.capitalize() # Capitaliza 'branca', 'amarela'


In [434]:
# 2.2 - Aplicação das funções para criar as novas colunas
df[['genero', 'raca']] = df.apply(consolidate_genero_raca_pair, axis=1)
df[['idade','escolaridade']] = df.apply(consolidate_idade_escolaridade, axis=1)
df[['senioridade', 'especialidade']] = df.apply(consolidate_especialidade_senioridade, axis=1)
df['raca_agrupada'] = df['raca'].apply(agrupar_raca)

In [435]:
# Exibe as novas colunas para ver o resultado
print("✅ Novas colunas de matrizes consolidadas com sucesso!")
print(df[['genero', 'raca', 'idade', 'escolaridade', 'senioridade', 'especialidade', 'criterios_vaga_afirmativa','raca_agrupada']].head(0))
print("\n")

✅ Novas colunas de matrizes consolidadas com sucesso!
Empty DataFrame
Columns: [genero, raca, idade, escolaridade, senioridade, especialidade, criterios_vaga_afirmativa, raca_agrupada]
Index: []




**Tratamento de Variáveis Ordinais (Escala Likert)**
Perguntas como impacto_vaga_afirmativa, sentimento_valorizacao, oportunidades_desenvolvimento e enfrentou_barreiras utilizam uma escala de concordância (Likert). Atualmente, elas estão como texto (object). Para análises futuras e para que os gráficos fiquem na ordem correta (de "Discordo" para "Concordo"), é uma boa prática convertê-las para um tipo de dado categórico ordenado.

In [436]:
from pandas.api.types import CategoricalDtype

# Definir a ordem correta das categorias
likert_order = [
    'Discordo Totalmente',
    'Discordo',
    'Neutro',
    'Concordo',
    'Concordo Totalmente'
]

# Criar um tipo categórico ordenado
cat_type = CategoricalDtype(categories=likert_order, ordered=True)

# Lista de colunas para aplicar a ordenação
likert_columns = [
    'impacto_vaga_afirmativa',
    'sentimento_valorizacao',
    'oportunidades_desenvolvimento',
    'enfrentou_barreiras'
]

# Aplicar a transformação
for col in likert_columns:
    # Ignoramos a categoria 'Não se aplica...' por enquanto
    # Você pode decidir como tratá-la depois (manter, remover, etc.)
    df[col] = df[col].astype(cat_type)

print("✅ Colunas de escala Likert convertidas para tipo categórico ordenado.")

✅ Colunas de escala Likert convertidas para tipo categórico ordenado.


In [437]:
# --- Passo 1: Pré-processamento e criação do dicionário de mapeamento ---
# Para garantir que a limpeza funcione, vamos padronizar todas as strings
# para minúsculas e sem acentos, além de remover espaços extras.
df['estado_limpo'] = df['estado'].str.lower().str.strip().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Dicionário de mapeamento com as variações que você forneceu.
estado_mapping = {
    'sao paulo': 'São Paulo',
    'sp': 'São Paulo',
    'sao': 'São Paulo',
    'pernambuco ': 'Pernambuco',
    'pernambuco': 'Pernambuco',
    'pr': 'Paraná',
    'parna - pr': 'Paraná',
    'parana': 'Paraná',
    'bahia': 'Bahia',
    'ceara': 'Ceará',
    'es': 'Espírito Santo',
    'minas gerais': 'Minas Gerais',
    'mg': 'Minas Gerais',
    'rio de janeiro': 'Rio de Janeiro',
    'rio de janeiro ': 'Rio de Janeiro',
    'pe': 'Pernambuco',
    'df': 'Distrito Federal',
    'rio grande do sul': 'Rio Grande do Sul',
    'sergipe': 'Sergipe',
    'santa catarina': 'Santa Catarina',
    'goias': 'Goiás',
    'mato grosso': 'Mato Grosso',
    'alagoas': 'Alagoas',
    'paraiba': 'Paraíba',
    'mato grosso do sul': 'Mato Grosso do Sul',
    'tocantins': 'Tocantins',
    'roraima': 'Roraima',
    'acre': 'Acre',
    'amazonas': 'Amazonas',
    'amapa': 'Amapá',
    'rondonia': 'Rondônia',
    'para': 'Pará',
    'maranhao': 'Maranhão',
    'piaui': 'Piauí',
    'rio grande do norte': 'Rio Grande do Norte'
}

# --- Passo 2: Aplicação do mapeamento ---
# A função .map() aplica as correções de forma eficiente.
# O .fillna(df_normalizado['estado_limpo']) garante que estados não listados
# no dicionário não sejam transformados em NaN, mantendo o valor original.
df['estado_normalizado'] = df['estado_limpo'].map(estado_mapping).fillna(df['estado_limpo'].str.title())

# --- Passo 3: Análise e comparação dos resultados ---
print("### Distribuição dos estados antes da normalização ###")
print(df['estado'].value_counts().to_string())
print("\n")

print("### Distribuição dos estados depois da normalização ###")
print(df['estado_normalizado'].value_counts().to_string())
print("\n")

# Remove a coluna original 'estado' e a coluna de apoio 'estado_limpo'
df.drop(columns=['estado'], inplace=True, errors='ignore')
# Renomeia a coluna normalizada para 'estado'
df.rename(columns={'estado_normalizado': 'estado'}, inplace=True)

print("✅ Normalização concluída. DataFrame atualizado.")

### Distribuição dos estados antes da normalização ###
estado
São Paulo            7
SP                   6
São Paulo            4
Bahia                4
Paraná               3
MG                   2
Pernambuco           1
Sao                  1
Pernambuco           1
ES                   1
Minas Gerais         1
PR                   1
Ceará                1
Rio Grande do Sul    1
Rio de Janeiro       1
Parná - PR           1
PE                   1
PERNAMBUCO           1
Rio de Janeiro       1
Sergipe              1
DF                   1


### Distribuição dos estados depois da normalização ###
estado_normalizado
São Paulo            18
Paraná                5
Pernambuco            4
Bahia                 4
Minas Gerais          3
Rio de Janeiro        2
Ceará                 1
Espírito Santo        1
Rio Grande do Sul     1
Sergipe               1
Distrito Federal      1


✅ Normalização concluída. DataFrame atualizado.


In [438]:
# 3.3 'fatores_permanencia'
print("3. Distribuição de 'estado' :")
if 'estado' in df.columns:
    fatores_permanencia_counts = df['estado'].value_counts(normalize=True).mul(100).round(1)
    print(fatores_permanencia_counts.to_string())
else:
    print("Column 'estado' not found in DataFrame.")
print("\n")


3. Distribuição de 'estado' :
estado
São Paulo            43.9
Paraná               12.2
Pernambuco            9.8
Bahia                 9.8
Minas Gerais          7.3
Rio de Janeiro        4.9
Ceará                 2.4
Espírito Santo        2.4
Rio Grande do Sul     2.4
Sergipe               2.4
Distrito Federal      2.4




In [439]:
df.describe()

,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,tempo_experiencia,dev_eng_software,devops,dados,suporte,infraestrutura,qa,si,ux_ui,po,negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_permanencia,opiniao_vagas_afirmativas,genero,raca,idade,escolaridade,senioridade,especialidade,raca_agrupada,estado_limpo,estado
count,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,27,41,41,41,41,17,41,41,41,41,41,41,41,41,41
unique,41,4,5,3,2,3,2,1,2,2,1,5,4,2,6,1,2,2,2,4,5,4,16,3,3,4,5,32,17,2,4,4,4,9,6,3,17,11
top,29/05/2025 14:25:27,,,,,,,,,,,1 a 3 anos,,,,,,,,,,,Não ingressei por vaga afirmativa,Concordo Totalmente,Concordo,Concordo Totalmente,Concordo,"Reconhecimento profissional, Promoção e progre...",Essenciais para melhorar a diversidade no trab...,Mulher(cis),preta,idade_26_35,Ensino Superior,dev_eng_software,Júnior,Negra,sao paulo,São Paulo
freq,1,29,19,37,38,20,29,41,39,35,41,18,28,40,32,41,40,40,39,36,37,36,13,18,19,16,16,3,1,40,21,22,18,13,17,33,11,18


In [411]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   senioridade                           100 non-null    object
 1   sentimento_opiniao_vagas_afirmativas  100 non-null    object
 2   sentimento_valorizacao                100 non-null    object
 3   area_atuacao                          100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [440]:
df.isnull()

,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,tempo_experiencia,dev_eng_software,devops,dados,suporte,infraestrutura,qa,si,ux_ui,po,negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_permanencia,opiniao_vagas_afirmativas,genero,raca,idade,escolaridade,senioridade,especialidade,raca_agrupada,estado_limpo,estado
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [441]:
df.head()

,timestamp,idade_ate_25,idade_26_35,idade_36_45,idade_46_ou_mais,preta,parda,indigena,amarela,branca,nao_respondeu_raca,tempo_experiencia,dev_eng_software,devops,dados,suporte,infraestrutura,qa,si,ux_ui,po,negocios,criterios_vaga_afirmativa,impacto_vaga_afirmativa,sentimento_valorizacao,oportunidades_desenvolvimento,enfrentou_barreiras,fatores_permanencia,opiniao_vagas_afirmativas,genero,raca,idade,escolaridade,senioridade,especialidade,raca_agrupada,estado_limpo,estado
0,29/05/2025 14:25:27,,,,Pós-graduação,,Mulher(cis),,,,,7 a 10 anos,,,,,,,,,,Estag/Trainee,Interseccional (direcionada a mulheres negras)...,Neutro,Neutro,Discordo,Discordo,Salário competitivo,None,Mulher(cis),parda,idade_46_ou_mais,Pós-graduação,negocios,Estag/Trainee,Negra,sao,São Paulo
1,30/05/2025 21:43:56,,Pós-graduação,,,Mulher(cis),,,,,,1 a 3 anos,Júnior,,,,,,,,,,Não ingressei por vaga afirmativa,NaN,Concordo Totalmente,Concordo Totalmente,Neutro,"Reconhecimento profissional, Remuneração e pac...",Essenciais para melhorar a diversidade no trab...,Mulher(cis),preta,idade_26_35,Pós-graduação,dev_eng_software,Júnior,Negra,pernambuco,Pernambuco
2,31/05/2025 12:03:02,,Ensino Superior,,,Mulher(cis),,,,,,Menos de 1 ano,Júnior,,,,,,,,,,Interseccional (direcionada a mulheres negras),Concordo,Concordo,Concordo,Discordo,"Desafios técnicos e projetos estimulantes, Rem...",None,Mulher(cis),preta,idade_26_35,Ensino Superior,dev_eng_software,Júnior,Negra,sao paulo,São Paulo
3,31/05/2025 17:40:18,,Ensino Superior,,,,Mulher(cis),,,,,Menos de 1 ano,Júnior,,,,,,,,,,"Raça (direcionada a pessoas negras), Gênero (d...",Concordo Totalmente,Concordo,Concordo Totalmente,Discordo,"Desafios técnicos e projetos estimulantes, Fle...",None,Mulher(cis),parda,idade_26_35,Ensino Superior,dev_eng_software,Júnior,Negra,sao paulo,São Paulo
4,02/06/2025 10:09:19,Ensino Médio,,,,Mulher(cis),,,,,,1 a 3 anos,,,,,,Estag/Trainee,,,,,Raça (direcionada a pessoas negras),Concordo,Concordo,Concordo Totalmente,Concordo,"Salário competitivo, Desafios técnicos e proje...",As vagas afirmativas são de extrema importânci...,Mulher(cis),preta,idade_ate_25,Ensino Médio,qa,Estag/Trainee,Negra,sao paulo,São Paulo


In [445]:
# Exemplo: Se você tem uma pasta chamada "TCC" e dentro dela uma subpasta "Dados"
caminho_da_pasta = '/content/drive/MyDrive/MBA/TCC/Materiais_e_Metodos/Dados/'
nome_do_arquivo = 'dados_limpos.csv'
caminho_completo = caminho_da_pasta + nome_do_arquivo

# Supondo que o seu DataFrame se chame 'df'
# (Crie um DataFrame de exemplo se estiver testando)
# df = pd.DataFrame({'coluna1': [1, 2], 'coluna2': ['A', 'B']})

# Passo 3: Salvar o DataFrame no caminho especificado
# O argumento 'index=False' é importante para não salvar o índice do DataFrame no CSV.
df.to_csv(caminho_completo, index=False)

print(f"Arquivo '{nome_do_arquivo}' salvo com sucesso em:")
print(caminho_da_pasta)

Arquivo 'dados_limpos.csv' salvo com sucesso em:
/content/drive/MyDrive/MBA/TCC/Materiais_e_Metodos/Dados/
